In [ ]:
#bibliotecas necssarias
!pip install google-cloud-speech
!pip install google-cloud-storage
!pip install google-auth
!pip install librosa

In [16]:

from google.cloud import speech, storage
from concurrent.futures import ThreadPoolExecutor
import os
import librosa
import numpy as np



os.environ["GOOGLE_APPLICATION_CREDENTIALS"]  = 'credenciais.json'
phrases_l=['Boa tarde Suzana', 'Bom dia Suzana','Bom dia'
           ]

def normalize_audio_remove_silence(file_path):
    y, sr = librosa.load(file_path, sr=9600)
    y_nom = librosa.util.normalize(y)

    energy = librosa.feature.rms(y=y_nom)
    threshold = np.median(energy)
    intervals = librosa.effects.split(y_nom, top_db=20, frame_length=512, hop_length=64)

    audio_seguiments =[y_nom[start:end] for start, end in intervals]


    audio_concat = np.concatenate(audio_seguiments)
    return audio_concat, sr

""" def get_sample_rate(file_path):
    with wave.open(file_path, 'rb') as wav_file:
        return wav_file.getframerate()
  """   
def transcribe_file(speech_file: str) : 
    """Transcribe the given audio file."""
    client = speech.SpeechClient()

   #sample_rate = get_sample_rate(speech_file)
    #print("sample_rate:", sample_rate)


    #with open(speech_file, "rb") as audio_file:
   #         content = audio_file.read()
    
    audio = speech.RecognitionAudio(uri=speech_file)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=9600,
        language_code="pt-BR", 
        model = 'phone_call',
        enable_separate_recognition_per_channel=True,
        enable_automatic_punctuation=True,
        speech_contexts=[speech.SpeechContext(phrases=phrases_l,boost=10.0)]
         
    )

    response = client.long_running_recognize(config=config, audio=audio)
    print("Aguarde operação em andamento...")
    response = response.result(timeout=90)
    transcript_builder = []
   
    for result in response.results:
        
        transcript_builder.append(f"\nTranscript: {result.alternatives[0].transcript}")
        transcript_builder.append(f"\nConfidence: {result.alternatives[0].confidence}")
    
    transcript = "".join(transcript_builder)
    #print(transcript)

    return transcript




In [15]:
transcribe_file('gs://repor_entrada_audio/entradas/teste.wav')

Aguarde operação em andamento...


'\nTranscript: Gimawa Laisa, boa tarde. Tudo bem também, você tá bem também eu mandar o orçamento novo e a previsão de faturamento, você me mandou no e-mail no e-mail agora no meu na minha mesa, eu vou tentar lá na mesa, daí vou ver aqui, tá bom? Tá bom. Obrigada, viu?\nConfidence: 0.9104512929916382'

In [3]:
# função de envio assincrono do arquivo

def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)
    
            
        blob.upload_from_filename(source_file_name)
        print(f"File uploaded to gs://{bucket_name}/{destination_blob_name}.")
    except Exception as e:
        print(f"Failed to upload {source_file_name} to {destination_blob_name}: {e}")

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")


def list_files_in_directory(directory):
    """Lists all files in a given directory."""
    return [os.path.join(directory, file) for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))]

def upload_files_to_gcs(bucket_name, directory):
    """Upload multiple files to GCS asynchronously from a directory."""
    files = list_files_in_directory(directory)
    with ThreadPoolExecutor() as executor:
        futures = []
        for source_file_name in files:
            destination_blob_name = os.path.basename(source_file_name)  # Use the file name as the blob name
            futures.append(executor.submit(upload_to_gcs, bucket_name, source_file_name, destination_blob_name))
        for future in futures:
            future.result()  # Wait for all uploads to complete




In [19]:
import soundfile as sf
#envio de arquivos 
diretorio_SN =False

# Nome do bucket GCS
bucket_name = "repor_entrada_audio"
source_file_name = "teste.wav"
#pasta e nomo do arquivo
destination_blob_name = f"entradas/{source_file_name}"
# Diretório local contendo os arquivos para upload
directory = ""
temp_audio_file ="temp_audio_file.wav"

audio_data, sample_rate= normalize_audio_remove_silence(source_file_name)

#audio_data_bytes = audio_data.astype(np.float32).tobytes()
sf.write(temp_audio_file, audio_data, sample_rate)

if diretorio_SN ==False:
    #bucket_name = enviar bucket  
    #source_file_name= nome do arquivo individual com extensão 
    #destination_blob_name= passa diretorio do BUCKET E NOME DO ARQUIVO

    upload_to_gcs(bucket_name, temp_audio_file, destination_blob_name)
else:
    #envio por diretorio 
    #bucket_name= enviar bucket 
    #directory = arquivos do diretorio local para se enviado para nuvem
    upload_files_to_gcs(bucket_name, directory)











File uploaded to gs://repor_entrada_audio/entradas/teste.wav.
File temp_audio_file.wav uploaded to entradas/teste.wav.


In [20]:
import wave

with wave.open("temp_audio_file.wav", "rb") as wav_file:
    sample_rate = wav_file.getframerate()
    print("Sample rate:", sample_rate)

Sample rate: 9600


In [13]:
audio_data, sample_rate= normalize_audio_remove_silence(source_file_name)

audio_data_bytes = audio_data.astype(np.float32).tobytes()


with open("teste_temp.wav", "wb") as f:
    f.write(audio_data)
